# IS319 - Deep Learning

## TP3 - Recurrent neural networks

Credits: Andrej Karpathy

The goal of this TP is to experiment with recurrent neural networks for a character-level language model to generate text that looks like training text data.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. Text data preprocessing

Several text datasets are provided, feel free to experiment with different ones throughout the TP. At the beginning, use a small subset of a given dataset (for example use only 10k characters).

In [2]:
text_data_fname = 'baudelaire.txt'  # ~0.1m characters (French)
# text_data_fname = 'proust.txt'      # ~7.3m characters (French)
# text_data_fname = 'shakespeare.txt' # ~0.1m characters (English)
# text_data_fname = 'lotr.txt'        # ~2.5m characters (English)
# text_data_fname = 'doom.c'          # ~1m characters (C Code)
# text_data_fname = 'linux.c'         # ~11.5m characters (C code)

text_data = open(text_data_fname, 'r').read()
text_data = text_data[:10000] # use a small subset
print(f'Dataset `{text_data_fname}` contains {len(text_data)} characters.')
print('Excerpt of the dataset:')
print(text_data[:2000])

Dataset `baudelaire.txt` contains 10000 characters.
Excerpt of the dataset:
LES FLEURS DU MAL

par

CHARLES BAUDELAIRE


AU LECTEUR


La sottise, l'erreur, le péché, la lésine,
Occupent nos esprits et travaillent nos corps,
Et nous alimentons nos aimables remords,
Comme les mendiants nourrissent leur vermine.

Nos péchés sont têtus, nos repentirs sont lâches,
Nous nous faisons payer grassement nos aveux,
Et nous rentrons gaîment dans le chemin bourbeux,
Croyant par de vils pleurs laver toutes nos taches.

Sur l'oreiller du mal c'est Satan Trismégiste
Qui berce longuement notre esprit enchanté,
Et le riche métal de notre volonté
Est tout vaporisé par ce savant chimiste.

C'est le Diable qui tient les fils qui nous remuent!
Aux objets répugnants nous trouvons des appas;
Chaque jour vers l'Enfer nous descendons d'un pas,
Sans horreur, à travers des ténèbres qui puent.

Ainsi qu'un débauché pauvre qui baise et mange
Le sein martyrisé d'une antique catin,
Nous volons au passage un plaisir c

**(Question)** Create a character-level vocabulary for your text data. Create two dictionaries: `ctoi` mapping each character to an index, and the reverse `itoc` mapping each index to its corresponding character. Implement the functions to convert text to tensor and tensor to text using these mappings. Apply these functions to some text data.

In [3]:
# Create the vocabulary and the two mapping dictionaries
# YOUR CODE HERE

import numpy as np

vocabulary = list(set(text_data))

print(vocabulary)

ctoi = {vocabulary[num]: num for num in range(0, len(vocabulary))}

itoc = {num : vocabulary[num] for num in range(0, len(vocabulary))}

print(itoc)
print(ctoi)

# Implement the function converting text to tensor
def text_to_tensor(text, ctoi):
    # YOUR CODE HERE
    tensor = np.array([ctoi[c] for c in text])
    tensor = torch.Tensor(tensor).int()

    return tensor 

# Implement the function converting tensor to text
def tensor_to_text(tensor, itoc):
    # YOUR CODE HERE
    tensor = tensor.numpy()
    text = [itoc[i] for i in tensor]

    return text

# Apply your functions to some text data
# YOUR CODE HERE
sample = text_data[:10]

print("Sample : " + str(sample))

s_tensor = text_to_tensor(sample, ctoi)
s_text = tensor_to_text(s_tensor, itoc)

print("Tensor of sample : " + str(s_tensor))
print("Text of tensor : " + "".join(s_text))

['a', 'f', ';', '«', '»', 'c', 'v', 'ô', 'É', 's', 'Q', 'R', 'V', 'o', 'N', 'n', 'u', 'â', 'm', '.', 'y', 'g', 'M', 'i', '-', 'ù', 'û', 'p', '?', 'h', 't', 'é', ',', ':', 'J', 'è', 'k', ' ', 'ê', 'î', 'C', 'b', 'z', 'l', 'P', 'q', 'x', 'B', 'D', 'U', 'H', 'ç', 'W', 'G', '_', 'A', 'I', '!', 'à', 'E', 'F', 'O', "'", 'S', 'T', '\n', 'd', 'j', 'r', 'e', 'L']
{0: 'a', 1: 'f', 2: ';', 3: '«', 4: '»', 5: 'c', 6: 'v', 7: 'ô', 8: 'É', 9: 's', 10: 'Q', 11: 'R', 12: 'V', 13: 'o', 14: 'N', 15: 'n', 16: 'u', 17: 'â', 18: 'm', 19: '.', 20: 'y', 21: 'g', 22: 'M', 23: 'i', 24: '-', 25: 'ù', 26: 'û', 27: 'p', 28: '?', 29: 'h', 30: 't', 31: 'é', 32: ',', 33: ':', 34: 'J', 35: 'è', 36: 'k', 37: ' ', 38: 'ê', 39: 'î', 40: 'C', 41: 'b', 42: 'z', 43: 'l', 44: 'P', 45: 'q', 46: 'x', 47: 'B', 48: 'D', 49: 'U', 50: 'H', 51: 'ç', 52: 'W', 53: 'G', 54: '_', 55: 'A', 56: 'I', 57: '!', 58: 'à', 59: 'E', 60: 'F', 61: 'O', 62: "'", 63: 'S', 64: 'T', 65: '\n', 66: 'd', 67: 'j', 68: 'r', 69: 'e', 70: 'L'}
{'a': 0, 'f'

## 2. Setup a character-level recurrent neural network

**(Question)** Setup a simple embedding layer with `nn.Embedding` to project character indices to `embedding_dim` dimensional vectors. Explain precisely how this layer works and what are its outputs for a given input sequence.

In [4]:
# YOUR CODE HERE

vocab_size = len(vocabulary)

embedding_dim = 5 # ?

embedding_layer = nn.Embedding(vocab_size, embedding_dim)

In [5]:
#Test 

sample = text_data[:10]

sample_tensor = text_to_tensor(sample, ctoi)

s_embedding = embedding_layer(sample_tensor)

print(s_embedding)

tensor([[ 1.0300, -1.5504, -0.4612,  0.2855,  1.0831],
        [-0.8769,  0.3289, -0.3306,  1.2520, -0.5546],
        [-1.3907,  0.1205, -0.3566,  1.3498, -0.9746],
        [-0.2978,  0.3296, -0.0816, -0.4782,  0.3649],
        [ 1.1182, -0.6595, -2.4232, -0.1074, -0.1791],
        [ 1.0300, -1.5504, -0.4612,  0.2855,  1.0831],
        [-0.8769,  0.3289, -0.3306,  1.2520, -0.5546],
        [-0.8367,  1.4818, -0.6446, -1.0677,  0.3188],
        [ 0.1404, -0.5862, -1.0873,  0.0299, -0.3499],
        [-1.3907,  0.1205, -0.3566,  1.3498, -0.9746]],
       grad_fn=<EmbeddingBackward0>)


YOUR ANSWER HERE

**(Question)** Setup a single-layer RNN with `nn.RNN` (without defining a custom class). Use `hidden_dim` size for hidden states. Explain precisely the outputs of this layer for a given input sequence.

In [6]:
# YOUR CODE HERE

hidden_dim = 3

simple_rnn = nn.Sequential(
    embedding_layer,  
    nn.RNN(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
)

In [9]:
sample_output, hid = simple_rnn(text_to_tensor(sample, ctoi))
print(sample_output)
print(hid)

tensor([[ 0.0813, -0.6228, -0.8158],
        [-0.4311,  0.5758,  0.3703],
        [-0.2812,  0.1575,  0.7780],
        [ 0.1292, -0.2155, -0.3440],
        [ 0.6797,  0.4672, -0.8148],
        [ 0.1272, -0.7725, -0.5920],
        [-0.3996,  0.5669,  0.3597],
        [ 0.3913,  0.1586, -0.0483],
        [ 0.4681, -0.0109, -0.2833],
        [-0.1605,  0.2112,  0.8182]], grad_fn=<SqueezeBackward1>)
tensor([[-0.1605,  0.2112,  0.8182]], grad_fn=<SqueezeBackward1>)


YOUR ANSWER HERE

**(Question)** Create a simple RNN model with a custom `nn.Module` class. It should contain: an embedding layer, a single-layer RNN, and a dense output layer. For each character of the input sequence, the model should predict the probability of the next character. The forward method should return the probabilities for next characters and the corresponding hidden states.
After completing the class, create a model and apply the forward pass on some input text. Understand and explain the results.

*Note:* depending on how you implement the loss function later, it can be convenient to return logits instead of probabilities, i.e. raw values of the output layer before any activation function. 

In [23]:
class CharRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        '''Initialize model parameters and layers.'''
        super().__init__()
        # YOUR CODE HERE
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        self.emb = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.lin = nn.Linear(hidden_dim, vocab_size)


    def forward(self, tensor_data, hidden_state=None):
        '''Apply the forward pass for some text data already converted to tensor.'''
        # YOUR CODE HERE
        out = self.emb(tensor_data)
        out, hid = self.rnn(out, hidden_state)
        out = self.lin(out)
        
        return out, hid

# Initialize a model and apply the forward pass on some input text
# YOUR CODE HERE

embedding_dim = 5
hidden_dim = 3

model = CharRNN(vocab_size, embedding_dim, hidden_dim)

sample = text_data[:10]

sample_tensor = text_to_tensor(sample, ctoi)

logits, hidden = model(sample_tensor)

probabilities = F.softmax(logits)

/tmp/ipykernel_20069/2784757363.py:37: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probabilities = F.softmax(logits)


YOUR ANSWER HERE

**(Question)** Implement a simple training loop to overfit on a small input sequence. The loss function should be a categorical cross entropy on the predicted characters. Monitor the loss function value over the iterations.

In [24]:
# Sample a small input sequence into tensor `input_seq` and store its corresponding expected sequence into tensor `target_seq`
# YOUR CODE HERE

# Implement a training loop overfitting an input sequence and monitoring the loss function
def train_overfit(model, input_seq, target_seq, n_iters=200, learning_rate=0.2):
    # YOUR CODE HERE
    criterion = nn.CrossEntropyLoss()
    # criterion = nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

    for i in range(n_iters):
        outputs, _ = model(input_seq)
        loss = criterion(outputs, target_seq)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if((i+1)%10==0):
            print('Epoch [{}/{}], Loss: {:.4f}'.format(i+1, n_iters, loss.item()))

    
# Initialize a model and make it overfit the input sequence
# YOUR CODE HERE

def one_hot(input_seq, target_seq, vocab_size, ctoi):
    one_hot_vec = np.zeros((len(input_seq), vocab_size))
    for i, c in enumerate(input_seq):
        t_index = ctoi[target_seq[i]]
        one_hot_vec[i][t_index] = 1
    return torch.Tensor(one_hot_vec)


input_seq = text_data[:100]
target_seq = text_data[1:101]

target_seq = one_hot(input_seq, target_seq, vocab_size, ctoi)
input_seq = text_to_tensor(input_seq, ctoi)

embedding_dim = 5
hidden_dim = 3

model = CharRNN(vocab_size, embedding_dim, hidden_dim)

train_overfit(model, input_seq, target_seq)

Epoch [10/200], Loss: 3.7675
Epoch [20/200], Loss: 3.1834
Epoch [30/200], Loss: 2.9093
Epoch [40/200], Loss: 2.7120
Epoch [50/200], Loss: 2.5866
Epoch [60/200], Loss: 2.4981
Epoch [70/200], Loss: 2.4327
Epoch [80/200], Loss: 2.3768
Epoch [90/200], Loss: 2.3247
Epoch [100/200], Loss: 2.2747
Epoch [110/200], Loss: 2.2272
Epoch [120/200], Loss: 2.1828
Epoch [130/200], Loss: 2.1407
Epoch [140/200], Loss: 2.1010
Epoch [150/200], Loss: 2.0626
Epoch [160/200], Loss: 2.0173
Epoch [170/200], Loss: 1.9835
Epoch [180/200], Loss: 1.9482
Epoch [190/200], Loss: 1.9211
Epoch [200/200], Loss: 1.8980


**(Question)** Implement a `predict_argmax` method for your `RNN` model. Then, verify your overfitting: use some characters of your input sequence as context to predict the remaining ones. Experiment with the current model and analyze the results.

In [38]:
class CharRNN(CharRNN):
    def predict_argmax(self, context_tensor, n_predictions):
        # Apply the forward pass for the context tensor
        # Then, store the last prediction and last hidden state
        # YOUR CODE HERE
        out, hid = self.forward(context_tensor)
        self.last_out = out[-1]
        self.last_hid = hid[-1].unsqueeze(-1)
        
        # Use the last prediction and last hidden state as inputs to the next forward pass
        # Do this in a loop to predict the next `n_predictions` characters
        # YOUR CODE HERE
        predictions = []
        for i in range(n_predictions):
            str_out = itoc[np.argmax(F.softmax(self.last_out, -1).detach().numpy())]
            predictions.append(str_out)
            self.last_out = text_to_tensor(str_out, ctoi)
            log, hid = self.forward(self.last_out, self.last_hid.view(1, self.hidden_dim))
            self.last_out = log
            self.last_hid = hid.unsqueeze(-1)  
            
        return predictions
            
        

# Initialize a model and make it overfit as above
# Then, verify your overfitting by predicting characters given some context
# YOUR CODE HERE

input_seq = text_data[:1000]
target_seq = text_data[1:1001]

target_seq = one_hot(input_seq, target_seq, vocab_size, ctoi)
input_seq = text_to_tensor(input_seq, ctoi)

embedding_dim = 10
hidden_dim = 30

model = CharRNN(vocab_size, embedding_dim, hidden_dim)

train_overfit(model, input_seq, target_seq)

predictions = model.predict_argmax(text_to_tensor("D'accord", ctoi), 200)

predictions = "".join(predictions)

print(predictions)

Epoch [10/200], Loss: 3.3685
Epoch [20/200], Loss: 2.9978
Epoch [30/200], Loss: 2.7820
Epoch [40/200], Loss: 2.6133
Epoch [50/200], Loss: 2.4889
Epoch [60/200], Loss: 2.3862
Epoch [70/200], Loss: 2.3005
Epoch [80/200], Loss: 2.2258
Epoch [90/200], Loss: 2.1601
Epoch [100/200], Loss: 2.1002
Epoch [110/200], Loss: 2.0445
Epoch [120/200], Loss: 1.9928
Epoch [130/200], Loss: 1.9446
Epoch [140/200], Loss: 1.9249
Epoch [150/200], Loss: 1.8735
Epoch [160/200], Loss: 1.8244
Epoch [170/200], Loss: 1.8640
Epoch [180/200], Loss: 1.7646
Epoch [190/200], Loss: 1.7174
Epoch [200/200], Loss: 1.6763
e le pas,
Et nous nous palt par cous des eut le pas,
Et nous nous palt par cous des eut le pas,
Et nous nous palt par cous des eut le pas,
Et nous nous palt par cous des eut le pas,
Et nous nous palt 


YOUR ANSWER HERE

Using the argmax function to predict the next character can yield a deterministic generator always predicting the same characters. Instead, it is common to predict the next character by sampling from the distribution of output predictions, adding some randomness into the generator.

**(Question)** Implement a `predict_proba` method for your `RNN` model. It should be very similar to `predict_argmax`, but instead of using argmax, it should randomly sample from the output predictions. To do that, you can use the `torch.distribution.Categorical` class and its `sample()` method. Verify that your method correctly added some randomness.

In [ ]:
class CharRNN(CharRNN):
    def predict_proba(self, input_context, n_predictions):
        # YOUR CODE HERE
        raise NotImplementedError()

# Verify that your predictions are not deterministic anymore
# YOUR CODE HERE
raise NotImplementedError()

## 3. Train the RNN model on text data

**(Question)** Adapt your previous code to implement a proper training loop for a text dataset. To do so, we need to specify a sequence length `seq_len`, acting similarly to the batch size in classic neural networks. Then, you can either randomly sample sequences of length `seq_len` from the text dataset over `n_iters` iterations, or properly loop over the text dataset for `n_epochs` epochs (with a random starting point for each epoch to ensure different sequences), to make sure the whole dataset is seen by the model. Feel free to adjust training and model parameters empirically. Start with a small model and a small subset of the text dataset, then move on to larger experiments. Remember to use GPU if available.

In [ ]:
# Create the text dataset, compute its mappings and convert it to tensor
# YOUR CODE HERE
raise NotImplementedError()

# Initialize training parameters
# YOUR CODE HERE
raise NotImplementedError()

# Initialize a character-level RNN model
# YOUR CODE HERE
raise NotImplementedError()

# Setup the training loop
# Regularly record the loss and sample from the model to monitor what is happening
# YOUR CODE HERE
raise NotImplementedError()

**(Question)** From your trained model, play around with its predictions: start with a custom input sequence and ask the model to predict the rest. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

## 4. Experiment with different RNN architectures

**(Question)** Experiment with different RNN architecures. Potential ideas are multi-layer RNNs, GRUs and LSTMs. All models can be extended to multi-layer using the `num_layers` parameter. Analyze and comment your results.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE